In [ ]:
# !pip install mysql-connector-python==9.0.0 Flask==3.0.3 pandas==2.2.2 flask-cors==4.0.1 requests==2.32.3

In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import matplotlib.pyplot as plt

In [3]:
# Columnas a utilizar
COLSTOUSE = [
  'periodo',
  'cole_area_ubicacion',
  'cole_naturaleza',
  'cole_cod_dane_establecimiento',
  'cole_depto_ubicacion',
  'cole_mcpio_ubicacion',
  'cole_nombre_establecimiento',
  'estu_depto_presentacion',
  'estu_mcpio_presentacion',
  'estu_fechanacimiento',
  'estu_estadoinvestigacion',
  'estu_genero',
  'estu_estudiante',
  'fami_estratovivienda'
  'desemp_ingles',
  'punt_ingles',
  'punt_matematicas',
  'punt_sociales_ciudadanas',
  'punt_c_naturales',
  'punt_lectura_critica',
  'punt_global'
]

# Detalles de configuracion de la base de datos
CONFIG = {
    'user': 'admin',
    'password': 'Unacontra123.',
    'host': 'icfes.cxa2yu08iduf.us-east-2.rds.amazonaws.com',
    'raise_on_warnings': True,
    'database': 'icfes_11'
}

In [24]:
def connect_to_database():
    try:
        connection = mysql.connector.connect(**CONFIG)
        if connection.is_connected():
            print("Conectado exitosamente a la base de datos")
            return connection
    except errorcode as e:
        print(f"Error al conectar a MySQL: {e}")
        return None

def execute_query(connection, query):
    cursor = connection.cursor(dictionary=True)
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except errorcode as e:
        print(f"Error al ejecutar la consulta: {e}")
        return None
    finally:
        cursor.close()

def get_promedios_colombia(connection, start=20142, end=20222):
    query = f"""
    SELECT
      *
    FROM promedios_colombia
    WHERE periodo >= {start} AND periodo <= {end}
    LIMIT 1000
    """
    return execute_query(connection, query)

def get_promedios_departamento(connection, start=20142, end=20222):
    query = f"""
    SELECT
      *
    FROM promedios_departamento
    WHERE periodo >= {start} AND periodo <= {end}
    LIMIT 1000
    """
    return execute_query(connection, query)

def get_promedios_colombia_year(connection, start=20142, end=20222):
    query = """
    SELECT
        LEFT(periodo, 4) AS year,
        CAST(AVG(promedio_global) AS DECIMAL(25,2)) AS promedio_global,
        CAST(AVG(promedio_lectura_critica) AS DECIMAL(25,2)) AS promedio_lectura_critica,
        CAST(AVG(promedio_matematicas) AS DECIMAL(25,2)) AS promedio_matematicas,
        CAST(AVG(promedio_c_naturales) AS DECIMAL(25,2)) AS promedio_c_naturales,
        CAST(AVG(promedio_sociales_ciudadanas) AS DECIMAL(25,2)) AS promedio_sociales_ciudadanas,
        CAST(AVG(promedio_ingles) AS DECIMAL(25,2)) AS promedio_ingles
    FROM 
        promedios_colombia
    WHERE periodo >=20142
    GROUP BY 
        LEFT(periodo, 4)
    ORDER BY 
        year
    LIMIT 1000
    """

    return execute_query(connection, query)

def get_promedios_departamento_year(connection, start=20142, end=20222):
    query = """
    SELECT
        LEFT(periodo, 4) AS year,
        departamento,
        CAST(AVG(promedio_global) AS DECIMAL(25,2)) AS promedio_global,
        CAST(AVG(promedio_lectura_critica) AS DECIMAL(25,2)) AS promedio_lectura_critica,
        CAST(AVG(promedio_matematicas) AS DECIMAL(25,2)) AS promedio_matematicas,
        CAST(AVG(promedio_c_naturales) AS DECIMAL(25,2)) AS promedio_c_naturales,
        CAST(AVG(promedio_sociales_ciudadanas) AS DECIMAL(25,2)) AS promedio_sociales_ciudadanas,
        CAST(AVG(promedio_ingles) AS DECIMAL(25,2)) AS promedio_ingles
    FROM 
        promedios_departamento
    WHERE periodo >=20142
    GROUP BY 
        departamento, 
        LEFT(periodo, 4)
    ORDER BY 
        year
    LIMIT 1000
    """

    return execute_query(connection, query)

def get_consulta_departamento(connection, departamento, start_year, end_year):
    # Consulta inicial por departamento y rango de años para saber 11
    start_periodo = start_year * 10 + 1
    end_periodo = end_year * 10 + 9

    query = f"""
    SELECT
    *
    FROM saber11
    WHERE cole_depto_ubicacion = "{departamento}" AND periodo >= {start_periodo} AND periodo <= {end_periodo}
    LIMIT 1000000;
    """
    return execute_query(connection, query)


def get_promedios_departamento_pro_year(connection, start=20181, end=20227):
    query = f"""
    SELECT
        LEFT(periodo, 4) AS year,
        departamento,
        CAST(AVG(promedio_c_ciudadana) AS DECIMAL(25,2)) AS promedio_c_ciudadana,
        CAST(AVG(promedio_comuni_escrita) AS DECIMAL(25,2)) AS promedio_comuni_escrita,
        CAST(AVG(promedio_ingles) AS DECIMAL(25,2)) AS promedio_ingles,
        CAST(AVG(promedio_lectura_critica) AS DECIMAL(25,2)) AS promedio_lectura_critica,
        CAST(AVG(promedio_razona_cuantitativo) AS DECIMAL(25,2)) AS promedio_razona_cuantitativo
    FROM 
        promedios_departamento_pro
    WHERE periodo >= {start} AND periodo <= {end}
    GROUP BY 
        departamento, 
        LEFT(periodo, 4)
    ORDER BY 
        year
    LIMIT 1000
    """

    return execute_query(connection, query)

def get_consulta_departamento_pro(connection, departamento, start_year, end_year):
    # Consulta inicial por departamento y rango de años para saber pro
    start_periodo = start_year * 10 + 1
    end_periodo = end_year * 10 + 9

    query = f"""
    SELECT
    *
    FROM saberpro_reduced
    WHERE inst_depto_ubicacion = "{departamento}" AND periodo >= {start_periodo} AND periodo <= {end_periodo}
    ORDER BY periodo
    LIMIT 1000000;
    """

    return execute_query(connection, query)

In [5]:
connection = connect_to_database()

Conectado exitosamente a la base de datos


In [19]:
query="""
    SELECT
    *
    FROM saber11
    WHERE cole_depto_ubicacion = SANTANDER AND periodo >= 20142 AND periodo <= 20152
    ORDER BY periodo
    LIMIT 1000000;
"""

result_df =  pd.DataFrame(execute_query(connection, query))

In [59]:
result_df = pd.DataFrame(get_consulta_departamento_pro(connection, "SANTANDER", 2018, 2018))

In [61]:
municipios = result_df['ESTU_INST_MUNICIPIO'].unique().tolist()
municipios

['BUCARAMANGA', 'SAN GIL', 'BARRANCABERMEJA', 'SOCORRO']

In [66]:
df = result_df[result_df['ESTU_INST_MUNICIPIO'] == 'BUCARAMANGA'].copy()
df.drop(columns=['ESTU_INST_MUNICIPIO'], inplace=True)
df.head(2)

,periodo,inst_depto_ubicacion,inst_naturaleza,inst_cod_institucion,ESTU_INST_DEPARTAMENTO,INST_NOMBRE_INSTITUCION,ESTU_DEPTO_PRESENTACION,ESTU_MCPIO_PRESENTACION,estu_fechanacimiento,ESTU_ESTADOINVESTIGACION,ESTU_GENERO,ESTU_ESTUDIANTE,FAMI_ESTRATOVIVIENDA,desemp_ingles,punt_ingles,punt_razona_cuantitativo,punt_c_ciudadana,punt_lectura_critica,punt_comuni_escrita
0,20183,SANTANDER,UNIVERSIDAD,2832,SANTANDER,UNIVERSIDAD DE SANTANDER - UDES-BUCARAMANGA,NORTE SANTANDER,CÚCUTA,02/02/1996,PUBLICAR,M,ESTUDIANTE,2,A1,138,153,108,147,151.0
1,20183,SANTANDER,INSTITUCIÓN UNIVERSITARIA,2847,SANTANDER,UNIVERSIDAD DE INVESTIGACION Y DESARROLLO - UD...,SANTANDER,BUCARAMANGA,23/04/1993,PUBLICAR,M,ESTUDIANTE,3,A1,127,164,127,140,192.0


In [67]:
PROMEDIOS_SABERPRO = ['promedio_ingles', 'promedio_razona_cuantitativo', 'promedio_c_ciudadana', 'promedio_lectura_critica', 'promedio_comuni_escrita']
PUNTAJES_SABERPRO = ['punt_ingles', 'punt_razona_cuantitativo', 'punt_c_ciudadana', 'punt_lectura_critica', 'punt_comuni_escrita']
df = result_df[['FAMI_ESTRATOVIVIENDA', *PUNTAJES_SABERPRO]].groupby('FAMI_ESTRATOVIVIENDA', as_index=False).mean().round(2).sort_values('FAMI_ESTRATOVIVIENDA')
df.columns = ['Estado', *PROMEDIOS_SABERPRO]

In [26]:
result = get_consulta_departamento(connection, "BOGOTA", 2014, 2022)
len(result)

712661

In [23]:
result_df = pd.DataFrame(result)
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712661 entries, 0 to 712660
Data columns (total 21 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   periodo                        712661 non-null  int64  
 1   cole_area_ubicacion            712661 non-null  object 
 2   cole_naturaleza                712661 non-null  object 
 3   cole_cod_dane_establecimiento  712661 non-null  int64  
 4   cole_depto_ubicacion           712661 non-null  object 
 5   cole_mcpio_ubicacion           712661 non-null  object 
 6   cole_nombre_establecimiento    712661 non-null  object 
 7   estu_depto_presentacion        712658 non-null  object 
 8   estu_mcpio_presentacion        712658 non-null  object 
 9   estu_fechanacimiento           712660 non-null  object 
 10  estu_estadoinvestigacion       712661 non-null  object 
 11  estu_genero                    712148 non-null  object 
 12  estu_estudiante               

In [21]:
result_df.__len__()

210007